# Ta feng Data

## Load all

In [1]:
from stat_data import Stat
import pickle

root_dir = "./data/ta_feng/"
filename = "transaction.dat"

f = open(root_dir+filename,"rb")
data = pickle.load(f)

f = open(root_dir+"train.pkl","rb")
train = pickle.load(f)

f = open(root_dir+"test.pkl","rb")
test = pickle.load(f)

f = open(root_dir+"validation.pkl","rb")
valid = pickle.load(f)

f = open(root_dir+"negative_sample.pkl","rb")
neg_train = pickle.load(f)

s = Stat(data)
trans = s.get_trans()
usrset = s.get_usrset()
itemset = s.get_itemset()

In [5]:
list(neg_train.keys())[0], neg_train[list(neg_train.keys())[0]]

(1835021,
 [{20360023,
   20392994,
   34000338108,
   37000323686,
   4710530997140,
   4712552800224,
   4714298609031,
   4718121002235,
   9771015996008},
  {20089917,
   4710177043118,
   4711093110342,
   4711770012051,
   4712297160201,
   4712566065398,
   4975060511222,
   8712000900045,
   9310022862601},
  {4710095987402,
   4711269680839,
   4711883060116,
   4712987110172,
   4714082600411,
   4715833517781,
   4718480800015},
  {20565206,
   4710762201046,
   4713243940991,
   4714756832070,
   4901872886609,
   4902831502417,
   8004800002624},
  {20519490,
   2100035002807,
   2270073000414,
   4710088432322,
   4712031000541,
   4714232210019,
   4714725703240,
   4714910110143,
   4716086062028,
   4934567511128},
  {4710043004946,
   4710063041204,
   4710677006651,
   4711629910361,
   4714581740922,
   4714957141988,
   4716758000389,
   4902431001747,
   4978955040968,
   8693357016354}])

In [64]:
s.show()


        # transactions: 61991
        # items       : 22326
        Ave item      : 8.6352857672888
        # users       : 10599
        


### Tuplize

In [79]:
if type(neg_train) is dict:
    neg_train = [(u, list(t)) for u, ts in neg_train.items() for t in ts]

In [75]:
ave = 0
for i in train:
    ave += len(i[1])
    
print(ave)

319088


In [ ]:
if type(train) is dict:
    train = [(u, list(t)) for u, ts in train.items() for t in ts]

In [31]:
%load_ext autoreload
%autoreload 2

import os
import random
import pickle
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# for reproducibility
seed=1234
def seed_everything(seed=1234):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
seed_everything(seed)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
root_dir = "./data/ta_feng/"
csv_file = "ta_feng_all_months_merged.csv"

df = pd.read_csv(root_dir+csv_file)

In [4]:
df

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
0,11/1/2000,1104905,45-49,115,110411,4710199010372,2,24,30
1,11/1/2000,418683,45-49,115,120107,4710857472535,1,48,46
2,11/1/2000,1057331,35-39,115,100407,4710043654103,2,142,166
3,11/1/2000,1849332,45-49,Others,120108,4710126092129,1,32,38
4,11/1/2000,1981995,50-54,115,100205,4710176021445,1,14,18
...,...,...,...,...,...,...,...,...,...
817736,2/28/2001,312790,35-39,114,530501,4713317035042,2,80,118
817737,2/28/2001,57486,40-44,115,530209,4710731060124,1,40,55
817738,2/28/2001,733526,>65,Unknown,510539,4716340052307,1,78,115
817739,2/28/2001,173704,45-49,115,520457,4714276145315,1,90,96


In [5]:
customer_id = set(df["CUSTOMER_ID"])

## Change Data structure
Data structure is \
[(customer id, transaction),...]

In [6]:
%time
import tqdm

data = []
trans = []  # for removing stop word items
n_trans = 0 # before removing fewer trans and item 
cnt = 0

for i in tqdm.tqdm(customer_id):
    customer = df[df["CUSTOMER_ID"] == i]
    trans_date = list(set(customer["TRANSACTION_DT"]))
    n_trans += len(trans_date)
    # print("custormer ID    :", i, "\n" \
    #       "transaction date:", trans_date, len(trans_date))

    for d in trans_date:
        product = customer[customer["TRANSACTION_DT"] == d]
        trans.append(list(product["PRODUCT_ID"]))
        # data.append((i, set(product["PRODUCT_ID"])))
        data.append([i, set(product["PRODUCT_ID"])])

  0%|          | 43/32266 [00:00<01:15, 426.72it/s]

CPU times: user 0 ns, sys: 1 µs, total: 1 µs
Wall time: 3.1 µs


100%|██████████| 32266/32266 [01:12<00:00, 446.36it/s]


## Transaction from 0 to 3

In [7]:
data[0:3]

[[1441802, {4714981010038}],
 [1441802, {4710549000039, 4712048022017}],
 [1703948,
  {20246037,
   20341992,
   2250271000683,
   4710049003608,
   4710063021091,
   4710088434500,
   4710094014765,
   4710094020179,
   4710114606048,
   4710126092730,
   4710162000126,
   4710207010226,
   4710221682782,
   4710357302905,
   4710363604000,
   4710363732000,
   4710466103080,
   4710570002606,
   4710626622857,
   4710908111116,
   4711045228101,
   4711713030531,
   4714499363053,
   4714957234147,
   4901301036056,
   4901301043979,
   4902430492416,
   4902430493024,
   4902511003418,
   4973271967807,
   8999118200100}]]

## Statics class for data stat

In [9]:
from stat_data import Stat
s = Stat(data)

### # transactions

In [10]:
s.n_data()

119578

### # items

In [11]:
itemset = s.get_itemset()
s.n_item()

23812

### # users

In [12]:
s.n_usr()

32266

### Average # item in a transaction

In [13]:
s.a_item()

6.8385572596966

## Preprocess

### 1. Remove stop word items

#### Count items

In [14]:
item_dct = {item : 0 for item in itemset}
for t in trans:
    for item in t:
        item_dct[item] += 1

#### Threshold

In [15]:
th = int(0.05*len(data))
th

5978

In [16]:
removed_items = {item for item, cnt in item_dct.items() if cnt>th}
print("The number of removed items:", len(removed_items), \
      "\nRemoved items \n", removed_items)

The number of removed items: 2 
Removed items 
 {4714981010038, 4711271000014}


#### Remove stop word item from itemset

In [17]:
print("Before remove:", len(itemset))
itemset -= removed_items
print("After remove :", len(itemset))

Before remove: 23812
After remove : 23810


#### Remove stop word item from transaction

In [18]:
for i in data:
    i[1] -= removed_items

### 2. Remove transactions which include less than 3 items

In [19]:
print(f"""
Before
# Data : {len(data)}
# Items: {len(itemset)}
""")

data = [(i[0], i[1]) for i in data if len(i[1]) > 2]
itemset = set()
for i in data:
    itemset |= i[1]

print(f"""
After
# Data : {len(data)}
# Items: {len(itemset)}
""")


Before
# Data : 119578
# Items: 23810


After
# Data : 85277
# Items: 23598



#### Make sure if items were removed

In [20]:
tmp = set()
for i in data:
    tmp |= i[1]

print(len(tmp), len(itemset))
assert len(tmp)==len(itemset)
del tmp

23598 23598


## 3. Remove users whose transactions are fewer than 3

#### Count # transactions per user

In [21]:
ucnt = {d[0] : 0 for d in data}

for i in data:
    ucnt[i[0]] += 1

#### Remove users

In [22]:
data = [(d[0], d[1]) for d in data if ucnt[d[0]]>2]

### Check Stat

In [23]:
s = Stat(data)
s.show()


        # transactions: 61991
        # items       : 22326
        Ave item      : 8.6352857672888
        # users       : 10599
        


## Convert list to tuple

In [24]:
data = [(i[0], i[1]) for i in data]

## Save dataset 

In [25]:
import pickle
root_dir = "./data/ta_feng/"
filename = "transaction.dat"

filehandler = open(root_dir+filename, "wb") 
pickle.dump(data, filehandler)

## Load dataset from pickle

In [4]:
import pickle
root_dir = "./data/ta_feng/"
filename = "transaction.dat"

f = open(root_dir+filename,"rb")
data = pickle.load(f)

In [5]:
s = Stat(data)
trans = s.get_trans()
usrset = s.get_usrset()
itemset = s.get_itemset()

## Make triple dataset for <br> BASKET-SENSITIVE FACTORIZATION MACHINE(BFM)

### Split dataset for train, test and validation based on user
Ratio is \
train : test : valid = 6 : 2 : 2

In [6]:
# dataset ratio
r_train = 0.6
r_test = 0.2
r_valid = 0.2

最初にuserがkeyでtransactionのlistがvalueの辞書型を作ってsplitする．\
(速度的に遅いかもしれない，データセットが大きくなったら遅くなるかも...)

#### Itemset per user(customer)
`usr_dct`はkeyがuserでvalueがそのuserが購入したitemの集合が入っている． \
だから，`itemset - usr_dct[usr]`でusrが購入していないアイテム集合を取得できる．これをNegative sampleを作るときに利用する．

In [7]:
import copy

usr_dct = {}
tmp = 0
for i in data:
    if not i[0] in usr_dct.keys():
         # idk why but if I don't use deepcopy, set in data is changed
        usr_dct[i[0]] = copy.deepcopy(i[1])
    else:
        usr_dct[i[0]] |= i[1]

n_usr = len(usr_dct.keys())
print(n_usr)

10599


In [8]:
udata = {usr : [] for usr in usr_dct.keys()}

for d in data:
    udata[d[0]].append(d[1])

Itemが2以下のtransactionがないかcheck

In [9]:
cnt = 0
for u, t in udata.items():
    if len(t) < 3: 
        cnt+=1
assert cnt==0, "Invalid transaction"

### Split based on user

In [35]:
import copy
import math

train = {usr:[] for usr in udata.keys()}
test = copy.deepcopy(train)
valid = copy.deepcopy(train)

for u,t in udata.items():
    l_t = len(t)
    if l_t == 3:
        train[u] = [copy.deepcopy(t[0])]
        test[u] = [copy.deepcopy(t[1])]
        valid[u] = [copy.deepcopy(t[2])]
    else:
        l_train = math.ceil(l_t*r_train)

        train[u] = copy.deepcopy(t[:l_train])

        l_test = math.ceil((l_t-l_train)*0.5)
        test[u] = copy.deepcopy(t[l_train:l_train+l_test])

        valid[u] = copy.deepcopy(t[l_train+l_test:])

calculate sum of data

In [36]:
cnt = 0

for tr, te, va in zip(train.values(), test.values(), valid.values()):
    cnt += (len(tr) + len(te) + len(va))

assert cnt==len(data), f"Invalid split, before split: {len(data)}, after split: {cnt}"

### Calculate pmi
We only need pmi for train dataset, so we calculated train dataset pmi with original dataset(valiable is `data`).

### Make negative sample
Negative sampleはPositive sampleをベースに作られている．\
1つのPositive sampleにつき2つ作る．つまりNegative sampleはPositive sampleの数の2倍になる．\
Negative sampleはベースとなっているアイテム数(|B|+v)と同じになっている．\
Negative sampleのアイテムはそのUserが購入していないアイテムで構成される．

In [37]:
import tqdm

neg_train = {usr:[] for usr in udata.keys()}
for usr, trans in tqdm.tqdm(train.items()):
    for t in trans:
        # item length
        l_item = len(t)

        # negative itemset
        neg_itemset = itemset - usr_dct[usr]
        neg_data = random.sample(neg_itemset, k=l_item*2)
        neg_train[usr].append(set(neg_data[:l_item]))
        neg_train[usr].append(set(neg_data[l_item:]))

100%|██████████| 10599/10599 [00:29<00:00, 364.32it/s]


In [38]:
l_train = 0
for trans in train.values():
    l_train += len(trans)

l_neg = 0
for trans in neg_train.values():
    l_neg += len(trans)

assert 2*l_train==l_neg, \
       f"""The number of negative sample should be 2*train.
                But got train sample:{l_train}, negative sample:{l_neg}"""

### Save train, test, validation and negative sample dataset

In [39]:
filehandler = open(root_dir+"train.pkl", "wb") 
pickle.dump(train, filehandler)

filehandler = open(root_dir+"test.pkl", "wb") 
pickle.dump(test, filehandler)

filehandler = open(root_dir+"validation.pkl", "wb") 
pickle.dump(valid, filehandler)

filehandler = open(root_dir+"negative_sample.pkl", "wb") 
pickle.dump(neg_train, filehandler)

### Load train, test, validation and negative sample

In [55]:
import pickle
root_dir = "./data/ta_feng/"

f = open(root_dir+"train.pkl","rb")
train = pickle.load(f)

f = open(root_dir+"test.pkl","rb")
test = pickle.load(f)

f = open(root_dir+"validation.pkl","rb")
valid = pickle.load(f)

f = open(root_dir+"negative_sample.pkl","rb")
neg_train = pickle.load(f)

## Convert data for model input using pytorch

In [32]:
import torch
import torch.nn.functional as F

Convert Itemset

In [33]:
itemset = list(itemset)
itemset.sort()
print(itemset.index(itemset[0]))

o_item = F.one_hot(torch.arange(0, len(itemset)))

0


Convert User

In [34]:
usrset = list(usrset)
usrset.sort()
print(usrset.index(usrset[0]))

o_usr = F.one_hot(torch.arange(0, len(usrset)))

0


In [8]:
o_item

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1]])

In [9]:
o_usr

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1]])

In [13]:
a = o_item[itemset.index(train[0][1][0])]
print("Tensor :", a)
print("Index  :", (a == 1).nonzero())

Tensor : tensor([0, 0, 0,  ..., 0, 0, 0])
Index  : tensor([[8526]])


### Convert train data structure to input format

#### <span style="color: red; ">Too much memory is used</span><pre>So, I have to use yeild to make generator.

In [14]:
len(train), type(train)

(37470, list)

In [15]:
random.Random(seed).sample(train, len(train))[:5]

[(1931600, [4718429100060, 4710058277021, 4710697581015]),
 (415484, [4710008111153, 4710549000756, 4710105020358]),
 (1705348,
  [4718433613952,
   4711863180070,
   4710291112172,
   4712076005204,
   4712076005341,
   4718433613150]),
 (540711,
  [4710247006562,
   4710060000099,
   4710110815208,
   4710008242123,
   4710114362029,
   4712031000015,
   4710467221523,
   4710114105046,
   4710467551224,
   4711763001017,
   4711985005626,
   7707950311516,
   4710114307037,
   4710046021407]),
 (1317954, [4719090790017, 4710088410610, 4710088414038])]

In [61]:
def convert(train, usrset, itemset, o_usr, o_item):
    train_input = []

    for d in train:
        usr = d[0]
        trans = d[1]
        basket = 0

        usr = o_usr[usrset.index(usr)]

        for item in trans:
            basket += o_item[itemset.index(item)]

        # print(trans)
        # print("items:", len(trans), basket.sum())
        # print("general basket\n", (basket==1).nonzero())
        for item in trans:
            # print(item)
            target = o_item[itemset.index(item)]
            t = basket-target
            # print("target \n", (target==1).nonzero())
            # print("t nonzero\n", (t==1).nonzero())
            t = torch.cat((target, t))
            # print(target.sum(), t[:target.shape[0]].sum(), t[target.shape[0]:].sum())
            t = torch.cat((usr, t))
            # train_input.append(t)
            yield t

In [26]:
tmp = convert(train, usrset, itemset, o_usr, o_item)

t = next(tmp)
print(t, (t==1).nonzero())
t = next(tmp)
print(t, (t==1).nonzero())
t = next(tmp)
print(t, (t==1).nonzero())
t = next(tmp)
print(t, (t==1).nonzero())


tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([[ 7620],
        [19125],
        [37412],
        [41429],
        [41433],
        [46964],
        [47304],
        [49077],
        [49467],
        [49478]])
tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([[ 7620],
        [15086],
        [41429],
        [41433],
        [41451],
        [46964],
        [47304],
        [49077],
        [49467],
        [49478]])
tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([[ 7620],
        [27152],
        [37412],
        [41429],
        [41433],
        [41451],
        [46964],
        [47304],
        [49077],
        [49467]])
tensor([0, 0, 0,  ..., 0, 0, 0]) tensor([[ 7620],
        [19107],
        [37412],
        [41429],
        [41451],
        [46964],
        [47304],
        [49077],
        [49467],
        [49478]])


In [63]:
%%time

import torch
import torch.nn.functional as F

def usr2vec(usrset):
    usrset = list(usrset)
    usrset.sort()
    o_usr = F.one_hot(torch.arange(0, len(usrset)))
    
    return usrset, o_usr

def item2vec(itemset):
    itemset = list(itemset)
    itemset.sort()
    o_item = F.one_hot(torch.arange(0, len(itemset)))
    
    return itemset, o_item

if type(train) is dict:
    train = [(u, list(t)) for u, ts in train.items() for t in ts]
    
usrset, o_usr = usr2vec(usrset)
itemset, o_item = item2vec(itemset)
ds = convert(train, usrset, itemset, o_usr, o_item)

cnt = 0
for d in ds:
    cnt += 1

print(cnt)
"""
    print(d)
    print("Index  :", (d == 1).nonzero())
    cnt += 1
    if cnt==4:
        break
"""

0
0
319088
CPU times: user 1min 49s, sys: 2.24 s, total: 1min 51s
Wall time: 1min 49s


'\n    print(d)\n    print("Index  :", (d == 1).nonzero())\n    cnt += 1\n    if cnt==4:\n        break\n'

319088
CPU times: user 1min 49s, sys: 2.24 s, total: 1min 51s
Wall time: 1min 49s

---
---
---
---

Preprocessing when store data

In [6]:
%time
data = []
trans = []  # for removing stop word items
n_trans = 0 # before removing fewer trans and item 
cnt = 0

for i in customer_id:
    customer = df[df["CUSTOMER_ID"] == i]
    trans_date = list(set(customer["TRANSACTION_DT"]))
    n_trans += len(trans_date)
    # print("custormer ID    :", i, "\n" \
    #       "transaction date:", trans_date, len(trans_date))

    # Remove users whose transactions are fewer than 3.
    if len(trans_date) < 4:
        cnt += 1
        continue

    for d in trans_date:
        product = customer[customer["TRANSACTION_DT"] == d]
        
        # Store only transactions which include more than 2 items.
        if len(set(product["PRODUCT_ID"])) > 2:
            trans.append(list(product["PRODUCT_ID"]))
            # data.append((i, set(product["PRODUCT_ID"])))
            data.append([i, set(product["PRODUCT_ID"])])
        else:
            cnt += 1

CPU times: user 0 ns, sys: 1 µs, total: 1 µs
Wall time: 3.1 µs


## split data using sklearn func (not based on user)

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
seed=1234

# Shuffle data
data = shuffle(data, random_state=seed)

# Split train, test and validation
train, test = train_test_split(data, test_size=0.4, random_state=seed)
test, valid = train_test_split(test, test_size=0.5, random_state=seed)

In [54]:
len(train), len(test), len(valid)

(37194, 12398, 12399)

---
---
---
---

#### Adjacency matrix

In [27]:
import tqdm

def make_adj(num_items, itemset, transactions):
    adjMatrix = np.zeros((num_items, num_items), dtype=np.float32)
    for t in tqdm.tqdm(transactions):
        for i, item1 in enumerate(t):
            # item1がtrain_itemsetに入ってなかったら飛ばす
            if not item1 in itemset: break
                
            sub_t = t[i+1:] # こっちはただの隣接行列
            idx1 = itemset.index(item1)
            for item2 in sub_t:
                # item2がtrain_itemsetに入ってなかったら飛ばす
                if not item2 in itemset: break
                idx2 = itemset.index(item2)
                adjMatrix[idx1][idx2] += 1

    return adjMatrix.astype(np.int32)

##### Train itemset

In [28]:
train_itemset = set()
for i in train:
    train_itemset |= i[1]

train_itemset = list(train_itemset)
print(len(train_itemset))
n_train_item = len(train_itemset)

19865


##### Whole transaction is `trans`

In [29]:
adj = make_adj(n_train_item, train_itemset, trans)

100%|██████████| 59191/59191 [19:38<00:00, 50.24it/s]  


##### Save adjacency matrix as numpy object

In [31]:
print(adj)
np.save(f"{root_dir}adj", adj)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [116]:
train[0]

(293044,
 {20570125,
  4710043100082,
  4710731070192,
  4712172200091,
  4714008271008,
  4719864067116,
  5998710132430,
  5998710132461})

In [105]:
# 内包表記のほうが速いっぽい
# ref: https://utgwkk.hateblo.jp/entry/2017/03/09/154314

# custormer id == 1835021
%time
custormer_id = 1835021
# target_custormer = lambda x: x[0]==custormer_id
# u_data = map(lambda x: x[1] if x[0]==custormer_id else None, trans_from_pickle)
# u_data = list(filter(target_custormer, trans_from_pickle))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [106]:
%time
target_tran = [i[1] for i in data if i[0]==custormer_id]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [107]:
target_tran

[{4710094004711,
  4710105002019,
  4710105031613,
  4710114361114,
  4710421090059,
  4710543282318,
  4711390436916,
  4713045519333,
  4715140032311,
  4716447022203},
 {3228021990293,
  4710088434593,
  4710105031613,
  4710114362029,
  4710121023302,
  4714220680091,
  4714220680107},
 {20554705,
  29000070295,
  4710105031613,
  4710205005750,
  4710731060124,
  4710740600090,
  4710740600106,
  8851954102126},
 {20412074, 4710063312168, 4710088433268},
 {4710015102946,
  4710431324236,
  4710431330282,
  4710431397056,
  4713691062900,
  4714005052013,
  4715190002999,
  4715833010893,
  4715833014310}]